<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/tcc/GSE120446_GSM3396179_from_gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSE120446_GSM3396179


This notebook uses the filtered count matrix resulting from the following google colab notebook. https://github.com/agalvezm/ACE2_scRNAseq/blob/master/notebooks_countmatrices/GSE120446_GSM3396179.ipynb Please run the notebook above and upload the results file in /content

In [ ]:
# define the values for the analysis

# accession id for the data
id = "GSE120446"
samp_id = ["GSM3396179"]

# If only bam available files, set bam = True, Fill link and filename
bam = True

# If fastq links available but are not ffq links
fastq_ffqlinks = False


if bam:
  # Assign link to python variable
  link_to_bam = "https://sra-pub-src-1.s3.amazonaws.com/SRR7881417/possorted_genome_bam_S1.bam.1"
  
  # Assigns the link to the bash variable BAM_LINK. To be used by wget
  %env BAM_LINK=$link_to_bam

  # Assign filename to python variable. Used to convert to fastq and remove bam file to fastq after conversion

  bam_filename="possorted_genome_bam_S1.bam.1"

if not bam:
  fastqs = ["",
            "",
            ]


env: BAM_LINK=https://sra-pub-src-1.s3.amazonaws.com/SRR7881417/possorted_genome_bam_S1.bam.1


In [ ]:
no_samples = 1

fastqs_per_sample = [4, 8, 8] 

sample_id = samp_id

database_id = [id] * no_samples

tissue = ["bone marrow"] * no_samples

cell_type = ["bone marrow mononuclear cells (BMMCs)"] * no_samples

condition = ["56 years old"] * no_samples

species = ["human"] * no_samples

technology = ["10xv2"] * no_samples

paper = ["Muus et al 2020"] * no_samples

figure = ["Fig 1 a,b  ED Fig 1 a,b,c,d  ED Fig 2 a,b,c,d,e"] * no_samples


# Set string variables for kb functions

species_kb = species[0]

technology_kb = technology[0]



# Imports and installs

In [ ]:
# install and import necessary software

# Install kb and scanpy
!pip -q install kb-python 
!pip -q install scanpy

import re
import os

# Setup

import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

     |████████████████████████████████| 35.4MB 118kB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 122kB 51.7MB/s 
     |████████████████████████████████| 112kB 47.2MB/s 
     |████████████████████████████████| 7.7MB 4.1MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 


# Downloads: (bam (if bam) and index

In [ ]:
if bam:

  # Install bamtofastq from 10x website (only bam files available)
  !wget http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
  !chmod +x bamtofastq-1.2.0
  # Download the bam file
  !wget -- continue ${BAM_LINK}




--2020-10-08 23:24:58--  http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.0.173, 104.18.1.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.0.173|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cf.10xgenomics.com/misc/bamtofastq-1.2.0 [following]
--2020-10-08 23:24:59--  https://cf.10xgenomics.com/misc/bamtofastq-1.2.0
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.0.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13288280 (13M) [binary/octet-stream]
Saving to: ‘bamtofastq-1.2.0’

bamtofastq-1.2.0    100%[===================>]  12.67M  44.9MB/s    in 0.3s    

2020-10-08 23:24:59 (44.9 MB/s) - ‘bamtofastq-1.2.0’ saved [13288280/13288280]

--2020-10-08 23:24:59--  http://continue/
Resolving continue (continue)... failed: Name or service not known.
wget: unable to resolve host address ‘continue’
-

In [ ]:
if bam:
  # Convert to fastq
  !./bamtofastq-1.2.0 --reads-per-fastq=500000000 $bam_filename ./fastqs\

  # Remove original bam file to save space
  !rm $bam_filename


bamtofastq v1.2.0
Args { arg_bam: "possorted_genome_bam_S1.bam.1", arg_output_path: "./fastqs", flag_nthreads: 4, flag_locus: None, flag_bx_list: None, flag_reads_per_fastq: 500000000, flag_gemcode: false, flag_lr20: false, flag_cr11: false }
Writing finished.  Observed 276118965 read pairs. Wrote 276118965 read pairs


In [ ]:
# Store fastq names on a list

if bam:
  # cd into fastqs folder
  %cd /content/fastqs

  #store the name of the folder generated by bamtofastq
  _filename = os.listdir()[0]

  # cd into that folder
  %cd $_filename

  # store fastq names in a list
  fastqs = os.listdir()


  # Remove I1 and R3 reads not relevant for our analysis

  # Initialize list containing elements to remove
  remov_elem = []

  print ("\n\nThis is the complete list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)

  # Search index (I1 or R3) fastqs and remove them from list
  for elem in fastqs:
    if re.search("_R3_", elem) or re.search("_I1_", elem):
      remov_elem = remov_elem +[elem]

  fastqs = [elem for elem in fastqs if elem not in remov_elem] 

  print ("\n\nThis is the filtered list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)


/content/fastqs
/content/fastqs/HFLNHBBXX_S1_MissingLibrary_1_HFLNHBBXX


This is the complete list of fastqs:
 -----------
bamtofastq_S1_L004_R1_001.fastq.gz
bamtofastq_S1_L002_R1_001.fastq.gz
bamtofastq_S1_L006_R2_001.fastq.gz
bamtofastq_S1_L005_R1_001.fastq.gz
bamtofastq_S1_L006_R1_001.fastq.gz
bamtofastq_S1_L005_I1_001.fastq.gz
bamtofastq_S1_L003_I1_001.fastq.gz
bamtofastq_S1_L004_R2_001.fastq.gz
bamtofastq_S1_L007_R2_001.fastq.gz
bamtofastq_S1_L002_I1_001.fastq.gz
bamtofastq_S1_L001_R2_001.fastq.gz
bamtofastq_S1_L006_I1_001.fastq.gz
bamtofastq_S1_L008_R1_001.fastq.gz
bamtofastq_S1_L004_I1_001.fastq.gz
bamtofastq_S1_L007_R1_001.fastq.gz
bamtofastq_S1_L007_I1_001.fastq.gz
bamtofastq_S1_L001_R1_001.fastq.gz
bamtofastq_S1_L002_R2_001.fastq.gz
bamtofastq_S1_L008_R2_001.fastq.gz
bamtofastq_S1_L003_R1_001.fastq.gz
bamtofastq_S1_L003_R2_001.fastq.gz
bamtofastq_S1_L005_R2_001.fastq.gz
bamtofastq_S1_L001_I1_001.fastq.gz
bamtofastq_S1_L008_I1_001.fastq.gz


This is the filtered list of fastq

In [ ]:
# Remove fastqs that wont be analyzed to save space
if bam:
  for elem in remov_elem:
    !rm $elem

In [ ]:
if bam:
  # sort fastqs alphabetically to get R1 and R2 in order
  fastqs = sorted(fastqs)

In [ ]:
# wget fastqs from non ffq links in fastqs folder
if not bam and not fastq_ffqlinks:
  !mkdir fastqs
  %cd fastqs
  for link in fastqs:
    !wget $link --continue

# update fastqs variable with name of files
fastqs = sorted(os.listdir())

In [ ]:
if bam:
  # Download the corresponding Kallisto index to folder containing fastqs
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam:
  %cd /content

  # Download the corresponding Kallisto index to content folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam and not fastq_ffqlinks:
  %cd /content/fastqs

  # Download the corresponding Kallisto index to fastq folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

[2020-10-09 00:20:20,434]    INFO Downloading files for human from https://caltech.box.com/shared/static/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz to tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz
[2020-10-09 00:21:35,878]    INFO Extracting files from tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz


In [ ]:
# Check to make sure the metadata is in the right order after sorting
print(fastqs)

['bamtofastq_S1_L001_R1_001.fastq.gz', 'bamtofastq_S1_L001_R2_001.fastq.gz', 'bamtofastq_S1_L002_R1_001.fastq.gz', 'bamtofastq_S1_L002_R2_001.fastq.gz', 'bamtofastq_S1_L003_R1_001.fastq.gz', 'bamtofastq_S1_L003_R2_001.fastq.gz', 'bamtofastq_S1_L004_R1_001.fastq.gz', 'bamtofastq_S1_L004_R2_001.fastq.gz', 'bamtofastq_S1_L005_R1_001.fastq.gz', 'bamtofastq_S1_L005_R2_001.fastq.gz', 'bamtofastq_S1_L006_R1_001.fastq.gz', 'bamtofastq_S1_L006_R2_001.fastq.gz', 'bamtofastq_S1_L007_R1_001.fastq.gz', 'bamtofastq_S1_L007_R2_001.fastq.gz', 'bamtofastq_S1_L008_R1_001.fastq.gz', 'bamtofastq_S1_L008_R2_001.fastq.gz']


# Process fastq files (modify kb command according to fastqs list)


In [ ]:
fastqs

['bamtofastq_S1_L001_R1_001.fastq.gz',
 'bamtofastq_S1_L001_R2_001.fastq.gz',
 'bamtofastq_S1_L002_R1_001.fastq.gz',
 'bamtofastq_S1_L002_R2_001.fastq.gz',
 'bamtofastq_S1_L003_R1_001.fastq.gz',
 'bamtofastq_S1_L003_R2_001.fastq.gz',
 'bamtofastq_S1_L004_R1_001.fastq.gz',
 'bamtofastq_S1_L004_R2_001.fastq.gz',
 'bamtofastq_S1_L005_R1_001.fastq.gz',
 'bamtofastq_S1_L005_R2_001.fastq.gz',
 'bamtofastq_S1_L006_R1_001.fastq.gz',
 'bamtofastq_S1_L006_R2_001.fastq.gz',
 'bamtofastq_S1_L007_R1_001.fastq.gz',
 'bamtofastq_S1_L007_R2_001.fastq.gz',
 'bamtofastq_S1_L008_R1_001.fastq.gz',
 'bamtofastq_S1_L008_R2_001.fastq.gz']

In [ ]:

if no_samples == 1:

  # Write the kb count command as a string with all fastqs of the list as an input
  cmd = "kb count --h5ad -i index.idx -g t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[0] + " "\
  + "--filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs) + "'"
  
  # Execute it
  !$cmd

# If more than one sample, iterate through fastqs accordingly
else:

  # Initializa counter for fastq files
  j = 0

  # Loop over samples for analysis
  for i in range(no_samples):

    fastqs_to_analyze = fastqs[j:j + fastqs_per_sample[i]]
    # Write the kb count command as a string
    cmd = "kb count --h5ad -i ../index.idx -g ../t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[i] + " \
    --filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs_to_analyze) + "'"

    # Execute it
    !$cmd

    # Update j to move to the next set of fastq
    j = j + fastqs_per_sample[i]






results = {}
tcc_results = {}
for sample in sample_id:
  # Read the filtered gene count matrix
  results["data_" + sample] = anndata.read("/content/result" + sample)


for sample in sample_id:
  output = "bus_raw_" + sample + "_from_gc"
  !mkdir $output
  folder_tcc = "tccoutput" + sample

  # read tcc matrix
  tcc_results[sample] = anndata.read(folder_tcc + "/counts_unfiltered/adata.h5ad")
  # apply gene count matrix's filter
  tcc_results[sample] = tcc_results[sample][results["data_" + sample].obs.index.values]
  # transfer obs data
  tcc_results[sample].obs = results["data_" + sample].obs
  # transfer metadata
  tcc_results[sample].uns = results["data_" + sample].uns
  # write tcc matrix
  tcc_results[sample].write("/content/tcc_" + sample)
  #gzip tcc matrix
  cmd = "gzip /content/tcc_" + sample
  !$cmd
  #gzip unfiltered bus file
  cmd = "gzip " + folder_tcc + "/output.unfiltered.bus"
  !$cmd
  # move and re-name bus file with sample id
  cmd = "mv " + folder_tcc + "/output.unfiltered.bus.gz " + output + "/" + sample +".unfiltered.bus.gz"
  !$cmd
  # gzip all unfiltered counts
  cmd = "gzip " + folder_tcc + "/counts_unfiltered/*"
  !$cmd

  # move unfiltered counts for both
  cmd = "mv " + folder_tcc + "/counts_unfiltered " + output + "/tcc_unfiltered"
  !$cmd

  cmd = "mv " + folder_tcc + "/*.json " + output 
  !$cmd
  # zip all files
  cmd = "zip -r "+ output + ".zip " + output
  !$cmd

[2020-10-09 00:22:16,415]    INFO Generating BUS file from
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L001_R1_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L001_R2_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L002_R1_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L002_R2_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L003_R1_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L003_R2_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L004_R1_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L004_R2_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L005_R1_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L005_R2_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L006_R1_001.fastq.gz
[2020-10-09 00:22:16,415]    INFO         bamtofastq_S1_L006_R2_001.fastq.gz
[2020-10-09 00:22

In [ ]:
import time
time.sleep(600000)